In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib as ta
import datetime, time
import pickle
from tqdm import tqdm_notebook

import pdblp as pb
import blpapi 

In [2]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [3]:
def save_file(dataframe, name):
    
    with open(name, 'wb') as f:
        pickle.dump(dataframe, f)

In [4]:
con = pb.BCon(port=8194, timeout=5000)
con.start()

In [4]:
df_code = pd.read_excel('data/etf_pair_code.xlsx', dtype={'symbol':str})

In [5]:
code_list = df_code.symbol.unique().tolist()

In [6]:
ticker_list = (df_code.symbol + ' ' + df_code.region).unique().tolist()

### test 1

In [ ]:
df = pd.DataFrame()
err_code = []
tic = time.perf_counter()
fields = ['HIGH', 'OPEN', 'LOW', 'VOLUME', 'PX_LAST']
cols = ['High', 'Open', 'Low', 'Volume', 'Adj Close']
start = '20100101'
end = '20190115'



for i in tqdm_notebook(ticker_list):
    try:
        temp_df = con.bdh(i, fields, start, end)
        temp_df = temp_df.loc[:, i]
        temp_df.columns = cols
        temp_df = temp_df.assign(code=i.split(' ')[0])

        try:
            df = pd.concat([df, temp_df], axis=0)
        except:
            print('concat data has problem, ticker code : %s' % i)

    except:
        print('retrieve %s data, have something wrong' % i)
        err_code.append(i)
toc = time.perf_counter()

print('retrieve data cost time: %s' % (toc-tic))

In [1]:
df.head()

In [2]:
df.tail()

In [9]:
df.to_csv('data/data.csv')

In [10]:
%load_ext autoreload
%autoreload 2

from utils import *

In [ ]:
result = calc_pair_trading('SMH', 
                           data_source='ol',
                           method='rolling', 
                           period=245, 
                           is_saving=True, 
                           save_name='rsi_sample')

如果上述代码没有问题，可以尝试以下代码

### test 2

In [17]:
intervals = np.linspace(0, len(code_list), 10, dtype=int)

In [ ]:
tic = time.perf_counter()

df = pd.DataFrame()

for i in tqdm_notebook(range(len(intervals)-1)):
    
    temp_list = ticker_list[intervals[i]:intervals[i+1]]
    
    temp_df = con.bdh(temp_list, fields, start, end)

    temp_df = pd.concat([temp_df[x].assign(code=x.split(' ')[0]).groupby(level=0, axis=1).first() for x in temp_df.columns.levels[0]], 
              axis=0,sort=False).reset_index().set_index(['date', 'code'])
    temp_df.columns = cols
    
    df = pd.concat([df, temp_df], axis=0)

toc = time.perf_counter()

print('retrieve data cost time: %s' % (toc-tic))

In [ ]:
# tic = time.perf_counter()

# df = con.bdh(ticker_list, fields, start, end)

# df = pd.concat([df[x].assign(code=x.split(' ')[0]).groupby(level=0, axis=1).first() for x in df.columns.levels[0]], 
#           axis=0,sort=False).reset_index().set_index(['date', 'code'])
# df.columns = cols

# toc = time.perf_counter()

# print('retrieve data cost time: %s' % (toc-tic))

In [1]:
df.to_csv('data/data.csv')

In [ ]:
result = calc_pair_trading('SMH', 
                           data_source='ol',
                           method='rolling', 
                           period=245, 
                           is_saving=True, 
                           save_name='rsi_sample')